In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

# Describe the functions

In [2]:
functions = [
    {
        "name": "get_item_info",
        "description": "Get name and price of a menu item of the chinese restaurant",
        "parameters": {
            "type": "object",
            "properties": {
                "item_name": {
                    "type": "string",
                    "description": "The name of the menu item, e.g. Chop-Suey",
                },
            },
            "required": ["item_name"],
        },
    },
    {
        "name": "place_order",
        "description": "Place an order for a menu item from the restaurant",
        "parameters": {
            "type": "object",
            "properties": {
                "item_name": {
                    "type": "string",
                    "description": "The name of the item you want to order, e.g. Chop-Suey",
                },
                "quantity": {
                    "type": "integer",
                    "description": "The number of items you want to order",
                    "minimum": 1
                },
                "address": {
                    "type": "string",
                    "description": "The address where the food should be delivered",
                },
            },
            "required": ["item_name", "quantity", "address"],
        },
    }
]


# Option 1: The LangChain Solution

We use create_openai_fn_chain to create a chain that handles the OpenAI Functions.

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.openai_functions import create_openai_fn_chain

In [4]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

template = """You are an AI chatbot having a conversation 
with a human.

Human: {human_input}
AI: """

prompt = PromptTemplate(
    input_variables=["human_input"], 
    template=template
)

chain = create_openai_fn_chain(
    functions, 
    llm, 
    prompt, 
    verbose=True
)

c:\users\johnd\appdata\local\programs\python\python39\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
c:\users\johnd\appdata\local\programs\python\python39\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `create_openai_fn_chain` was deprecated in LangChain 0.1.1 and will be removed in 0.2.0. Use create_openai_fn_runnable instead.
  warn_deprecated(


As we can see below, this app appears to work well:

In [5]:
chain.run("How much does Chop-Suey cost?")

c:\users\johnd\appdata\local\programs\python\python39\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
You are an AI chatbot having a conversation 
with a human.

Human: How much does Chop-Suey cost?
AI: 

> Finished chain.


{'arguments': {'item_name': 'Chop-Suey'}, 'name': 'get_item_info'}

In [6]:
chain.run("I want to order two Chop-Suey to 321 Street")



> Entering new LLMChain chain...
Prompt after formatting:
You are an AI chatbot having a conversation 
with a human.

Human: I want to order two Chop-Suey to 321 Street
AI: 

> Finished chain.


{'arguments': {'item_name': 'Chop-Suey',
  'quantity': 2,
  'address': '321 Street'},
 'name': 'place_order'}

### Problem: this app breaks if the user makes an off-topic question

In [7]:
# chain.run("How old did my grandfather get?") # Not-so-good error message.

# Option 2: Solution using the OpenAI API

In [7]:
!pip install openai

You should consider upgrading via the 'c:\users\johnd\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [8]:
import openai

def chat(query):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[{"role": "user", "content": query}],
        functions=functions
    )
    message = response.choices[0].message
    return message

In [9]:
chat("I want to order two Chop-Suey to 321 Street")

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"item_name":"Chop-Suey","quantity":2,"address":"321 Street"}', name='place_order'), tool_calls=None)

In [10]:
chat("Who is the current Pope?")

ChatCompletionMessage(content="I'm sorry, I can't provide real-time information. You can look up the current Pope online for the most up-to-date information. Is there anything else I can assist you with?", role='assistant', function_call=None, tool_calls=None)

In [14]:
message = chat("I want to order two Chop-Suey to 321 Street")
message

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"address":"321 Street","item_name":"Chop-Suey","quantity":2}', name='place_order'), tool_calls=None)

In [16]:
print(type(message))

if (hasattr(message, 'function_call')): # not a real dictionary
    print("function call needed!")

<class 'openai.types.chat.chat_completion_message.ChatCompletionMessage'>
function call needed!


# Let's use a dictionary as a fake database to check full functionality

In [22]:
fake_db = {
    "items": {
        "Chop-Suey": {"price": 15.00, "ingredients": ["chop", "suey", "cheese"]},
        "Lo-Main": {"price": 10.00, "ingredients": ["lo", "main", "basil"]},
        "Chin-Gun": {"price": 12.50, "ingredients": ["chin", "gunu", "tomato sauce"]},
        "Won-Ton": {"price": 11.00, "ingredients": ["won", "ton", "mushrooms"]},
    },
    "orders": []
}

## Let's create the functions

In [17]:
def get_item_info(item_name):
    item = fake_db["items"].get(item_name)
    
    if not item:
        return f"No information available for item: {item_name}"

    return {"name": item_name, "price": item["price"], "ingredients": item["ingredients"]}

def place_order(item_name, quantity, address):
    if item_name not in fake_db["items"]:
        return f"We don't have {item_name}!"
    
    if quantity < 1:
        return "You must order at least one item."
    
    order_id = len(fake_db["orders"]) + 1
    order = {
        "order_id": order_id,
        "item_name": item_name,
        "quantity": quantity,
        "address": address,
        "total_price": fake_db["items"][item_name]["price"] * quantity
    }

    fake_db["orders"].append(order)
    
    return f"Order placed successfully! Your order ID is {order_id}. Total price is ${order['total_price']}."


## Now let's check the App

In [18]:
message = chat("I want to order two Chop-Suey to 321 Street")
message

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"item_name":"Chop-Suey","quantity":2,"address":"321 Street"}', name='place_order'), tool_calls=None)

In [20]:
import json

print(message.function_call.name)
print(json.loads(message.function_call.arguments))


place_order
{'item_name': 'Chop-Suey', 'quantity': 2, 'address': '321 Street'}


In [23]:
function_name = message.function_call.name
arguments = json.loads(message.function_call.arguments)

response = place_order(**arguments)
print(response)

Order placed successfully! Your order ID is 1. Total price is $30.0.


## Let´s improve the app making it dynamic (the LLM can continue the conversation after the order has been placed)

In [25]:
import openai
import json

class ChatBot:
    
    def __init__(self, database):
        self.fake_db = database
        
    def chat(self, query):
        initial_response = self.make_openai_request(query)
        
        message = initial_response.choices[0].message
        
        if (hasattr(message, 'function_call') & (message.function_call != None)):
            function_name = message.function_call.name
            arguments = json.loads(message.function_call.arguments)
            function_response = getattr(self, function_name)(**arguments)
            
            follow_up_response = self.make_follow_up_request(query, message, function_name, function_response)
            return follow_up_response.choices[0].message.content
        else:
            return message.content
    
    def make_openai_request(self, query):
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo-0613",
            messages=[{"role": "user", "content": query}],
            functions=functions
        )
        return response

    def make_follow_up_request(self, query, initial_message, function_name, function_response):
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": query},
                initial_message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )
        return response

    def place_order(self, item_name, quantity, address):
        if item_name not in self.fake_db["items"]:
            return f"We don't have {item_name}!"
        
        if quantity < 1:
            return "You must order at least one item."
        
        order_id = len(self.fake_db["orders"]) + 1
        order = {
            "order_id": order_id,
            "item_name": item_name,
            "quantity": quantity,
            "address": address,
            "total_price": self.fake_db["items"][item_name]["price"] * quantity
        }

        self.fake_db["orders"].append(order)
        
        return f"Order placed successfully! Your order ID is {order_id}. Total price is ${order['total_price']}."

    def get_item_info(self, item_name):
        if item_name in self.fake_db["items"]:
            item = self.fake_db["items"][item_name]
            return f"Item: {item['name']}, Price: ${item['price']}"
        else:
            return f"We don't have information about {item_name}."

database = {
    "items": {
        "Chop-Suey": {
            "name": "Chop-Suey",
            "price": 15.0
        },
        "Lo-Mein": {
            "name": "Lo-Mein",
            "price": 12.0
        }
    },
    "orders": []
}


### Let's check the app

In [29]:
bot = ChatBot(database=database)
response = bot.chat("I want to order two Chop-Suey to 321 Street")

In [30]:
response

'Your order for two Chop-Suey to be delivered at 321 Street has been successfully placed. The order ID is 2, and the total price is $30.0.'

## Let's ask for a type of food that is not in the menu

In [31]:
response = bot.chat("I want to order one spring roll to 321 Street")
response

'I apologize, but it seems that we do not have spring rolls available. Is there anything else you would like to order?'

In [32]:
print(database)

{'items': {'Chop-Suey': {'name': 'Chop-Suey', 'price': 15.0}, 'Lo-Mein': {'name': 'Lo-Mein', 'price': 12.0}}, 'orders': [{'order_id': 1, 'item_name': 'Chop-Suey', 'quantity': 2, 'address': '321 Street', 'total_price': 30.0}, {'order_id': 2, 'item_name': 'Chop-Suey', 'quantity': 2, 'address': '321 Street', 'total_price': 30.0}]}


## Let's ask an off-topic question

In [28]:
response= bot.chat("Who is the current Pope?")
response

'The current Pope is Pope Francis.'

## As you see, the app does not break now.